<a href="https://colab.research.google.com/github/Masayoriosan/project/blob/main/project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


# Load data
pwt10 = pd.read_stata('https://dataverse.nl/api/access/datafile/354098')
# Filter and select relevant columns
data = pwt10.loc[pwt10['country'].isin(["France","Germany","Canada","Italy","Japan","United Kingdom","United States"])][['year', 'countrycode', 'rgdpna', 'rkna', 'emp', 'labsh']]
data = data.loc[(data['year'] >= 1995) & (data['year'] <= 2019)].dropna()
# Calculate additional columns
data['a'] = 1 - data['labsh']  # 資本の所得割合（Capital's Share of Income）
data['KL'] = (data['rkna']**data['a']) * (data['emp']**(1 - data['a']))
data['A'] = data['rgdpna'] / ((data['rkna']**data['a']) * (data['emp']**(1 - data['a'])))  # 総要素生産性（Total Factor Productivity, TFP）
data['Y'] = data['rgdpna']  # 国内総生産（Gross Domestic Product, GDP）


data['logy'] = np.log(data['rgdpna'] / data['emp'])  # Log GDP per worker
data['logk'] = np.log(data['KL'] / data['emp'])  # Log Capital per worker
data['logA'] = np.log(data['A'])




# Calculate time differences
data['dt'] = data['year'].diff()  # Time difference


# Calculate differentials
data['dlogy_dt'] = data['logy'].diff() / data['dt']
data['dlogk_dt'] = data['logk'].diff() / data['dt']
data['dlogA_dt'] = data['logA'].diff() / data['dt']


# 年度でソート
data = data.sort_values('year')


# ISOコードでグループ化
grouped_data = data.groupby('countrycode')


# TFP成長率と資本深化率を計算
data['TFP_growth'] = data['dlogA_dt'] * 100  # TFP成長率
data['capital_deepening'] = data['dlogk_dt'] * 100  # 資本深化率
data['GDP_growth'] = data['dlogy_dt'] * 100  # 経済成長率（GDP成長率）






# 欠損値を削除
data = data.dropna()


# サマリー統計量を計算
summary = data.groupby('countrycode').agg({'GDP_growth': 'mean',
                                           'capital_deepening': 'mean',
                                           'TFP_growth': 'mean'})


# TFP寄与率と資本深化寄与率を計算
summary['TFP寄与率'] = summary['TFP_growth'] / summary['GDP_growth'] * 100
summary['資本深化寄与率'] = summary['capital_deepening'] / summary['GDP_growth'] * 100




# 結果を表示
print(summary)


             GDP_growth  capital_deepening  TFP_growth       TFP寄与率  \
countrycode                                                           
CAN            0.887700           0.239437    0.648264    73.027397   
DEU            0.662448           0.161122    0.501324    75.677399   
FRA            0.816542           0.603822    0.212720    26.051325   
GBR            1.036837           1.311810   -0.274969   -26.520018   
ITA           -0.045964           0.503274   -0.549236  1194.914917   
JPN            0.757615           0.036508    0.721106    95.181152   
USA            1.440635           0.605465    0.835166    57.972073   

                 資本深化寄与率  
countrycode               
CAN            26.972775  
DEU            24.322161  
FRA            73.948730  
GBR           126.520309  
ITA         -1094.921265  
JPN             4.818804  
USA            42.027660  
